In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
from functools import reduce

In [2]:
def join_election_to_candidate(df):
    mod_df = df.copy()
    for i in range(2, len(df.columns)):
        mod_df[((df.iloc[:, i].name).replace("\n", "__")+"___"+df.iloc[:,0].name).upper()] = df[(df.iloc[:, i].name)]
        mod_df = mod_df.drop(columns=[(df.iloc[:, i].name)])
        try:
            mod_df = mod_df.drop(columns=[df.iloc[:,0].name])
        except:
            continue
    return mod_df


file_numbers = [i for i in range(0,56)]
total_county_data = []
base_df = pd.DataFrame(columns=["Precinct", "county_from_file"])
for file_number in file_numbers:
    sheet_count = len(pd.read_excel("./raw-from-source/mt_county_prec_er/CTYALL Results (" + str(file_number)+").xlsx", sheet_name=None).keys())
    within_county_data = []
    for sheet_number in range(0,sheet_count):
        df = pd.read_excel("./raw-from-source/mt_county_prec_er/CTYALL Results (" + str(file_number)+").xlsx", sheet_name=sheet_number, header=6)
        df = join_election_to_candidate(df)
        df["county_from_file"] = str(file_number+1)
        df["Precinct"] = df["Precinct"].astype(str).str.upper().str.replace("#", "").str.replace("PRECINCT", "").str.replace("' ", "")
        df["Precinct_UNQ"] = df["county_from_file"] + "-:-" + df["Precinct"]
        df.drop(["county_from_file", "Precinct"], axis = 1, inplace = True)
        within_county_data.append(df)
    merged_county_data = reduce(lambda x, y: pd.merge(x, y, on = "Precinct_UNQ", how = "outer"), within_county_data)
    merged_county_data.reset_index(inplace = True, drop = True)
    total_county_data.append(merged_county_data)
    
combined = pd.concat(total_county_data)
combined = combined[~combined['Precinct_UNQ'].str.contains("TOTALS")]
combined['Precinct_UNQ'].nunique()
#In 2020 files looks like there were 663 precincts, so this seems reasonable - https://github.com/nonpartisan-redistricting-datahub/pdv-mt/blob/main/vest-mt-2020/vest-mt-2020-validation.ipynb

663

In [3]:
base_df = combined

## Grabbing info for column dictionaries

In [4]:
#Get party
potential_party = base_df.columns.str.split(pat="__").str[1].unique()
party_dict = {'DEMOCRAT':'D','LIBERTARIAN':'L','REPUBLICAN':'R','INDEPENDENT':'I','NON-PARTISAN':'N','GREEN':'G','YES':'Y','NO':'N'}

#Get last name
potential_last_names = base_df.columns.str.split(pat="__").str[0].str.split(pat=" ").str[-1].unique()
last_name_dict = pd.Series(potential_last_names.str.slice(stop=3).values, index=potential_last_names).to_dict()

#Get office
general_office_dict = {"UNITED STATES REPRESENTATIVE": "CON", "STATE REP": "SL", "STATE SEN":"SU", 
               "PUBLIC SERVICE": "PSC", "SUPREME COURT": "SSC", "LEGISLATIVE REFERENDUM":"REF", "CONSTITUTIONAL AMENDMENT":"CA"}

#Get CONG DIST
cong_dist = base_df.columns[base_df.columns.str.contains("UNITED")].str.split(pat="___").str[-1].str.split(pat="\n").str[-1].str.slice(stop=1).unique()

#Get SLDL DIST
sldl_dist = base_df.columns[base_df.columns.str.contains("STATE REP")].str.split(pat="___").str[-1].str.split(pat=" ").str[-1].unique()

#GET SLDU DIST
sldu_dist = base_df.columns[base_df.columns.str.contains("STATE SEN")].str.split(pat="___").str[-1].str.split(pat="DISTRICT ").str[1]

#GET PSC DIST
psc_dist = base_df.columns[base_df.columns.str.contains("PUBLIC SERVICE")].str.split(pat="___").str[-1].str.split("DISTRICT ").str[-1]

#GET MT SUPREME COURT DIST
ssc_dist = base_df.columns[base_df.columns.str.contains("SUPREME COURT")].str.split("JUSTICE #").str[-1]

#GET LEG REF #
leg_ref_num = base_df.columns[base_df.columns.str.contains("REFERENDUM")].str.split(pat="___").str[-1].str.split(pat="NO. ").str[-1]

#GET CONST AMENDMENT #
ca_num = base_df.columns[base_df.columns.str.contains("CONST")].str.split(pat="___").str[-1].str.split(pat="NO. ").str[-1]

### Mapping columns
- Make df with column names (all) inside
- make column for office, name, party fill with .contains() and then dictionary key values

In [5]:
dfcol = pd.DataFrame(base_df.columns, columns=['og_column'])
#SET PARTY
dfcol['party_code'] = 'na'
dfcol.loc[dfcol['og_column'].str.split(pat="__").str[1].isin(party_dict), "party_code"] = dfcol['og_column'].str.split(pat="__").str[1].map(party_dict)
#SET NAME
dfcol['last_name_code'] = 'na'
dfcol.loc[dfcol['og_column'].str.split(pat="__").str[0].str.split(pat=" ").str[-1].isin(last_name_dict), "last_name_code"] = dfcol['og_column'].str.split(pat="__").str[0].str.split(pat=" ").str[-1].map(last_name_dict)
#SET OFFICE
dfcol['office_code'] = 'na'
dfcol.loc[dfcol['og_column'].str.contains("UNITED"), "office_code"] = "CON"
dfcol.loc[dfcol['og_column'].str.contains("STATE REP"), "office_code"] = "SL"
dfcol.loc[dfcol['og_column'].str.contains("STATE SEN"), "office_code"] = "SU"
dfcol.loc[dfcol['og_column'].str.contains("PUBLIC SERVICE"), "office_code"] = "PSC"
dfcol.loc[dfcol['og_column'].str.contains("SUPREME COURT"), "office_code"] = "SSC"
dfcol.loc[dfcol['og_column'].str.contains("REFERENDUM"), "office_code"] = "REF"
dfcol.loc[dfcol['og_column'].str.contains("CONST"), "office_code"] = "A"

#SET CONG DIST
dfcol['cong_dist'] = 'na'
dfcol.loc[dfcol['og_column'].str.contains("UNITED"), "cong_dist"] = dfcol['og_column'][dfcol['og_column'].str.contains("UNITED")].str.split(pat="___").str[-1].str.split(pat="\n").str[-1].str.slice(stop=1)
dfcol.loc[dfcol['og_column'].str.contains("STATE REP"), "sl_dist"] = dfcol['og_column'][dfcol['og_column'].str.contains("STATE REP")].str.split(pat="___").str[-1].str.split(pat=" ").str[-1]
dfcol.loc[dfcol['og_column'].str.contains("STATE SEN"), "su_dist"] = dfcol['og_column'][dfcol['og_column'].str.contains("STATE SEN")].str.split(pat="___").str[-1].str.split(pat="DISTRICT ").str[1]
dfcol.loc[dfcol['og_column'].str.contains("PUBLIC SERVICE"), "psc_dist"] = dfcol['og_column'][dfcol['og_column'].str.contains("PUBLIC SERVICE")].str.split(pat="___").str[-1].str.split("DISTRICT ").str[-1]
dfcol.loc[dfcol['og_column'].str.contains("SUPREME COURT"), "ssc_dist"] = dfcol['og_column'][dfcol['og_column'].str.contains("SUPREME COURT")].str.split("JUSTICE #").str[-1]
dfcol.loc[dfcol['og_column'].str.contains("REFERENDUM"), "ref_no"] = dfcol['og_column'][dfcol['og_column'].str.contains("REFERENDUM")].str.split(pat="___").str[-1].str.split(pat="NO. ").str[-1]
dfcol.loc[dfcol['og_column'].str.contains("CONST"), "amend_no"] = dfcol['og_column'][dfcol['og_column'].str.contains("CONST")].str.split(pat="___").str[-1].str.split(pat="NO. ").str[-1]

new_col = 'na'
#CONG - leaving as single char dist # because only 2 total
dfcol.loc[dfcol["office_code"]=="CON", "new_col"] = "G" + dfcol["office_code"] + dfcol['cong_dist'] + dfcol['party_code'] + dfcol['last_name_code']

#sl and slu
dfcol.loc[dfcol["office_code"]=="SL", "new_col"] = "G" + dfcol["office_code"] + dfcol['sl_dist'].str.zfill(3) + dfcol['party_code'] + dfcol['last_name_code']

dfcol.loc[dfcol['og_column'].str.contains("UNEX"), "su_dist"] = dfcol['su_dist'].str.slice(stop = 2)
dfcol.loc[dfcol["office_code"]=="SU", "new_col"] = "G" + dfcol["office_code"] + dfcol['su_dist'].str.zfill(2) + dfcol['party_code'] + dfcol['last_name_code']

#psc
dfcol.loc[dfcol["office_code"]=="PSC", "new_col"] = "G22" + dfcol["office_code"] + dfcol['psc_dist'] + dfcol['party_code'] + dfcol['last_name_code']

#ssc - list something other than party??
dfcol.loc[dfcol["office_code"]=="SSC", "new_col"] = "G22" + dfcol["office_code"] + + dfcol['og_column'].str.slice(start=-1) + dfcol['last_name_code']

#ref
dfcol.loc[dfcol["office_code"]=="REF", "new_col"] = "G22" + dfcol["office_code"] + dfcol['ref_no'] + dfcol['last_name_code']

#const amend
dfcol.loc[dfcol["office_code"]=="A", "new_col"] = "G22" + dfcol["office_code"] + dfcol['amend_no'] + dfcol['last_name_code']

#Other cols
dfcol.loc[dfcol['og_column']=="Precinct_UNQ", "new_col"] = "UNIQUEID"
dfcol.loc[dfcol['og_column']=="Precinct", "new_col"] = "PRECINCT"
dfcol.loc[dfcol['og_column']=="count_from_file", "new_col"] = "county_file_id"

dfcol_cut = dfcol[dfcol['new_col'].isna()]
dfcol = dfcol[~dfcol['new_col'].isna()]

dfcol["office_code"].value_counts()

SL     180
SU      47
CON      7
SSC      4
PSC      3
A        2
REF      2
na       1
Name: office_code, dtype: int64

In [6]:
dfcol[dfcol["office_code"]=="na"]

,og_column,party_code,last_name_code,office_code,cong_dist,sl_dist,su_dist,psc_dist,ssc_dist,ref_no,amend_no,new_col
4,Precinct_UNQ,na,Pre,na,na,NaN,NaN,NaN,NaN,NaN,NaN,UNIQUEID


In [7]:
dfcol.shape

(246, 12)

In [8]:
col_dict = pd.Series(dfcol['new_col'].values,index=dfcol['og_column']).to_dict()
final = combined[dfcol["og_column"]]
final = final.rename(columns = col_dict)
final = final.fillna(0)

#Check for column with no votes
for col in final.columns:
    if final[col].sum() == 0:
        print(col)

In [9]:
final["UNIQUEID"]

0        1-:- 1
1       1-:- 10
2       1-:- 14
3       1-:- 15
4       1-:- 17
        ...    
39    56-:-55.4
40    56-:-55.5
41    56-:-56.1
42    56-:-56.2
43    56-:-56.3
Name: UNIQUEID, Length: 663, dtype: object

In [10]:
county_fips_dict = {"1" :"3", "2" :"1","3" :"5","4" :"7","5" :"9","6" :"11","7" :"13","8" :"15","9" :"17","10" :"19","11" :"21","12" :"23","13" :"25","14" :"27",
"15" :"29","16" :"31","17" :"33","18" :"35","19" :"37","20" :"39","21" :"41","22" :"43","23" :"45","24" :"47","25" :"49","26" :"51","27" :"53","28" :"55",
"29" :"57","30" :"59","31" :"61","32" :"63","33" :"65","34" :"67","35" :"69","36" :"71","37" :"73","38" :"75","39" :"77","40" :"79","41" :"81","42" :"83",
"43" :"85","44" :"87","45" :"89","46" :"91","47" :"93","48" :"95","49" :"97","50" :"99","51" :"101","52" :"103","53" :"105","54" :"107","55" :"109","56" :"111"}

final['cty_file_number'] = final["UNIQUEID"].str.split("-:").str[0]
final["COUNTYFP"] = final["cty_file_number"].map(county_fips_dict)
final["PRECINCT"] = final["UNIQUEID"].str.split(":-").str[1]

In [11]:
#Make floats int
final[list(set(final.columns)-{'UNIQUEID','PRECINCT','cty_file_number','COUNTYFP'})]=final[list(set(final.columns)-{'UNIQUEID','PRECINCT','cty_file_number','COUNTYFP'})].astype(int)
#Reorder
election_cols = final.columns[final.columns.str.startswith("G")].sort_values()
#Sort alphabetically
final = final[['UNIQUEID','PRECINCT','COUNTYFP']+list(election_cols)]
final

,UNIQUEID,PRECINCT,COUNTYFP,G22A48NO,G22A48YES,G22PSC1RPIN,G22PSC5DREP,G22PSC5RBUK,G22REF131NO,G22REF131YES,G22SSC1DAL,G22SSC1RIC,G22SSC2BRO,G22SSC2GUS,GCON1DTRA,GCON1LLAM,GCON1RZIN,GCON2DRON,GCON2IBUC,GCON2LRAN,GCON2RROS,GSL001RGUN,GSL002RDUR,GSL003DGET,GSL003RMIT,GSL004DPIN,GSL004RREG,GSL005DFER,GSL005RBEN,GSL006RREG,GSL007DKEN,GSL007RSPR,GSL008LDAO,GSL008RFAL,GSL009RBRO,GSL010RKEE,GSL011RSMI,GSL012DLAU,GSL012RREK,GSL013DHIN,GSL013RFIE,GSL014RLOG,GSL015DJR,GSL015RFOS,GSL016DWOL,GSL017DSPO,GSL017RFIT,GSL018RJON,GSL019RMIN,GSL020DRIS,GSL020RAND,GSL021DGRA,GSL021RBUT,GSL022DHEN,GSL022LROS,GSL022RSHE,GSL023DSMI,GSL023RKER,GSL024DBES,GSL024RGAL,GSL025DKRO,GSL025RGIS,GSL026DROS,GSL026RNIK,GSL027RKAS,GSL028DTUS,GSL028RHIL,GSL029RFLA,GSL030DPAL,GSL030RBER,GSL031DSMI,GSL031RGOR,GSL032DBOY,GSL033DOPH,GSL033RKNU,GSL034RKNU,GSL035RLER,GSL036RPHA,GSL037RSCH,GSL038DMUG,GSL038RKME,GSL039RPAR,GSL040ROBL,GSL041DPEN,GSL041RGRE,GSL042DPER,GSL042RMCD,GSL043RSEE,GSL044DSMI,GSL044RBRE,GSL045RZOL,GSL046DWAR,GSL046RMER,GSL047DBAU,GSL047RMAD,GSL048DMER,GSL048RETC,GSL049DKER,GSL049RWYL,GSL050DREA,GSL050RSTR,GSL051DBOE,GSL051RYAK,GSL052DMAR,GSL052RESS,GSL053RNIC,GSL054RMOO,GSL055RDEM,GSL056RVIN,GSL057DFLA,GSL057RNAV,GSL058DGRE,GSL058RBAR,GSL059DWEI,GSL059RMAL,GSL060DBIS,GSL060RSKA,GSL061DHAM,GSL061GKEL,GSL061RBOW,GSL062DSTA,GSL062RGRE,GSL063DBUC,GSL063RPUR,GSL064DGRI,GSL064LCAM,GSL064RGIL,GSL065DKOR,GSL065RCOC,GSL066DMAT,GSL067DMAR,GSL067RHIN,GSL068DHAN,GSL068RHIN,GSL069DHAM,GSL069LJR,GSL069RCAR,GSL070DJAC,GSL070RDOO,GSL071RWAL,GSL072DGIB,GSL072RWEL,GSL073DLYN,GSL073RFRE,GSL074DHAR,GSL074RKEP,GSL075RBER,GSL076DHAW,GSL076RNOR,GSL077DNOV,GSL077RFIT,GSL078RFRA,GSL079DSMI,GSL079RPIG,GSL080RBEA,GSL081DROM,GSL081RSAR,GSL082DCAF,GSL082RTON,GSL083DABB,GSL083RLEA,GSL084DCOH,GSL084RRUC,GSL085DSTO,GSL085RBIN,GSL086DBRO,GSL086RBED,GSL087IMOO,GSL087RMAR,GSL088DMOU,GSL088RRUS,GSL089DSUL,GSL089RWAN,GSL090DMAR,GSL090GMAR,GSL090LHIN,GSL090RAUL,GSL091DKEO,GSL091RWAN,GSL092DSTE,GSL092RHOP,GSL093DAZZ,GSL093LBRA,GSL093RREA,GSL094DFRA,GSL094RMAP,GSL095DHOW,GSL095LWIN,GSL095RSUB,GSL096DKAR,GSL096RWHI,GSL097DJAC,GSL097RHEL,GSL098DCAR,GSL098LARM,GSL098RSHE,GSL099DTHA,GSL099RDAR,GSL100DZEP,GSL100LVAN,GSL100RMCC,GSU01RCUF,GSU04DWAT,GSU04RFUL,GSU05RNOL,GSU08DWEB,GSU08RJEN,GSU09RGIL,GSU11DJAC,GSU11REMR,GSU12DBAC,GSU12RMCK,GSU13DSCH,GSU13RTRE,GSU14DBRE,GSU14RTEM,GSU15DPAL,GSU15RBAR,GSU19RBOG,GSU20RUSH,GSU22DDEN,GSU22RZOL,GSU24DKEL,GSU24RTID,GSU27RLEN,GSU29RMAN,GSU30DISA,GSU30RESP,GSU32DFLO,GSU32RCHA,GSU33DHAY,GSU33RDEW,GSU34DLYN,GSU34RVAN,GSU39DMUL,GSU39RVER,GSU41DELL,GSU41RGAL,GSU42DDUN,GSU42ROLS,GSU43DSCH,GSU43RELL,GSU48DMOR,GSU48RHOR,GSU49DCUR,GSU49RTSC,GSU50DOLS,GSU50RKNO
0,1-:- 1,1,3,18,99,0,0,0,56,66,17,85,63,58,0,0,0,29,22,2,76,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,43,84,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1-:- 10,10,3,8,45,0,0,0,28,24,12,36,37,15,0,0,0,5,10,1,37,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,44,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1-:- 14,14,3,36,61,0,0,0,23,71,29,66,52,47,0,0,0,44,8,1,48,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,75,29,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

### Check file
Manually compare via statewide results from: https://sosmt.gov/elections/results/

In [12]:
from op_verification import reference_data
check = final.copy()
check['county'] = "30"+check['COUNTYFP'].str.zfill(3)
check["COUNTYNM"] = check['county'].map(reference_data.geoid_to_county_name)

In [13]:
#County fips assignment is correct - not the cause of the swap compared to official results
check[['COUNTYNM', "COUNTYFP"]][check['COUNTYNM']=="McCone County"]

,COUNTYNM,COUNTYFP
0,McCone County,55
1,McCone County,55
2,McCone County,55
3,McCone County,55
4,McCone County,55
5,McCone County,55
6,McCone County,55
7,McCone County,55


In [14]:
check[['COUNTYNM', "COUNTYFP"]][check['COUNTYNM']=="Madison County"]

,COUNTYNM,COUNTYFP
0,Madison County,57
1,Madison County,57
2,Madison County,57


In [15]:
#Match
check[check.columns[check.columns.str.contains("CON")]].sum()

GCON1DTRA    115265
GCON1LLAM      9593
GCON1RZIN    123102
GCON2DRON     43480
GCON2IBUC     47195
GCON2LRAN      3018
GCON2RROS    121979
dtype: int64

In [16]:
#Match except Madison and McCone swapped
check[check.columns[(check.columns.str.contains("CON"))|(check.columns.str.contains("COUNTY"))]].groupby('COUNTYNM').sum()

,GCON1DTRA,GCON1LLAM,GCON1RZIN,GCON2DRON,GCON2IBUC,GCON2LRAN,GCON2RROS
COUNTYNM,,,,,,,
Beaverhead County,1330,193,3004,0,0,0,0
Big Horn County,0,0,0,1214,505,51,1476
Blaine County,0,0,0,587,208,20,1040
Broadwater County,0,0,0,399,559,55,2417
Carbon County,0,0,0,1030,1420,89,3294
Carter County,0,0,0,37,46,13,663
Cascade County,0,0,0,6638,6397,410,15634
Chouteau County,0,0,0,395,390,42,1463
Custer County,0,0,0,848,684,70,2959


In [17]:
#Match
check[check.columns[check.columns.str.contains("PSC")]].sum()

G22PSC1RPIN    56347
G22PSC5DREP    41572
G22PSC5RBUK    54078
dtype: int64

In [18]:
#Match except Madison County votes should be in McCone county
check[check.columns[(check.columns.str.contains("PSC"))|(check.columns.str.contains("COUNTY"))]].groupby('COUNTYNM').sum()

,G22PSC1RPIN,G22PSC5DREP,G22PSC5RBUK
COUNTYNM,,,
Beaverhead County,0,0,0
Big Horn County,0,0,0
Blaine County,1371,0,0
Broadwater County,0,0,0
Carbon County,0,0,0
Carter County,0,0,0
Cascade County,20665,0,0
Chouteau County,1785,0,0
Custer County,0,0,0


In [19]:
#Match
check[check.columns[(check.columns.str.contains("SSC"))]].sum()

G22SSC1DAL     89008
G22SSC1RIC    307270
G22SSC2BRO    201159
G22SSC2GUS    239219
dtype: int64

In [20]:
#Match except Madison and McCone votes are swapped
check[check.columns[(check.columns.str.contains("SSC"))|(check.columns.str.contains("COUNTY"))]].groupby('COUNTYNM').sum()

,G22SSC1DAL,G22SSC1RIC,G22SSC2BRO,G22SSC2GUS
COUNTYNM,,,,
Beaverhead County,624,3251,2461,1869
Big Horn County,774,2098,1420,1733
Blaine County,304,1276,834,900
Broadwater County,661,2369,1973,1295
Carbon County,1204,3756,2850,2750
Carter County,120,470,484,175
Cascade County,5136,19610,12395,14936
Chouteau County,360,1571,1193,943
Custer County,770,3226,2421,1981


In [21]:
#Note I cut out "Shall Judge XXX of XXX, XXX of state of Montana be retained in office for another term?" Because not in Ballotopedia, but included in this source's statewide results.

In [22]:
#Match
check[check.columns[(check.columns.str.contains("A48"))]].sum()

G22A48NO      78334
G22A48YES    365091
dtype: int64

In [23]:
#Match except Madison and McCone votes are swapped 
check[check.columns[(check.columns.str.contains("A48"))|(check.columns.str.contains("COUNTY"))]].groupby('COUNTYNM').sum()

,G22A48NO,G22A48YES
COUNTYNM,,
Beaverhead County,784,3561
Big Horn County,852,2280
Blaine County,313,1468
Broadwater County,543,2780
Carbon County,1058,4540
Carter County,115,594
Cascade County,5000,22901
Chouteau County,441,1738
Custer County,740,3701


In [24]:
#Match
check[check.columns[(check.columns.str.contains("REF131"))]].sum()

G22REF131NO     235904
G22REF131YES    213001
dtype: int64

In [25]:
#Match except Madison and McCone votes are swapped 
check[check.columns[(check.columns.str.contains("REF131"))|(check.columns.str.contains("COUNTY"))]].groupby('COUNTYNM').sum()

,G22REF131NO,G22REF131YES
COUNTYNM,,
Beaverhead County,2066,2341
Big Horn County,1301,1752
Blaine County,756,983
Broadwater County,1218,2073
Carbon County,2847,2690
Carter County,188,531
Cascade County,13666,14434
Chouteau County,929,1208
Custer County,1968,2489


### Export File

In [26]:
check.head(2)

,UNIQUEID,PRECINCT,COUNTYFP,G22A48NO,G22A48YES,G22PSC1RPIN,G22PSC5DREP,G22PSC5RBUK,G22REF131NO,G22REF131YES,G22SSC1DAL,G22SSC1RIC,G22SSC2BRO,G22SSC2GUS,GCON1DTRA,GCON1LLAM,GCON1RZIN,GCON2DRON,GCON2IBUC,GCON2LRAN,GCON2RROS,GSL001RGUN,GSL002RDUR,GSL003DGET,GSL003RMIT,GSL004DPIN,GSL004RREG,GSL005DFER,GSL005RBEN,GSL006RREG,GSL007DKEN,GSL007RSPR,GSL008LDAO,GSL008RFAL,GSL009RBRO,GSL010RKEE,GSL011RSMI,GSL012DLAU,GSL012RREK,GSL013DHIN,GSL013RFIE,GSL014RLOG,GSL015DJR,GSL015RFOS,GSL016DWOL,GSL017DSPO,GSL017RFIT,GSL018RJON,GSL019RMIN,GSL020DRIS,GSL020RAND,GSL021DGRA,GSL021RBUT,GSL022DHEN,GSL022LROS,GSL022RSHE,GSL023DSMI,GSL023RKER,GSL024DBES,GSL024RGAL,GSL025DKRO,GSL025RGIS,GSL026DROS,GSL026RNIK,GSL027RKAS,GSL028DTUS,GSL028RHIL,GSL029RFLA,GSL030DPAL,GSL030RBER,GSL031DSMI,GSL031RGOR,GSL032DBOY,GSL033DOPH,GSL033RKNU,GSL034RKNU,GSL035RLER,GSL036RPHA,GSL037RSCH,GSL038DMUG,GSL038RKME,GSL039RPAR,GSL040ROBL,GSL041DPEN,GSL041RGRE,GSL042DPER,GSL042RMCD,GSL043RSEE,GSL044DSMI,GSL044RBRE,GSL045RZOL,GSL046DWAR,GSL046RMER,GSL047DBAU,GSL047RMAD,GSL048DMER,GSL048RETC,GSL049DKER,GSL049RWYL,GSL050DREA,GSL050RSTR,GSL051DBOE,GSL051RYAK,GSL052DMAR,GSL052RESS,GSL053RNIC,GSL054RMOO,GSL055RDEM,GSL056RVIN,GSL057DFLA,GSL057RNAV,GSL058DGRE,GSL058RBAR,GSL059DWEI,GSL059RMAL,GSL060DBIS,GSL060RSKA,GSL061DHAM,GSL061GKEL,GSL061RBOW,GSL062DSTA,GSL062RGRE,GSL063DBUC,GSL063RPUR,GSL064DGRI,GSL064LCAM,GSL064RGIL,GSL065DKOR,GSL065RCOC,GSL066DMAT,GSL067DMAR,GSL067RHIN,GSL068DHAN,GSL068RHIN,GSL069DHAM,GSL069LJR,GSL069RCAR,GSL070DJAC,GSL070RDOO,GSL071RWAL,GSL072DGIB,GSL072RWEL,GSL073DLYN,GSL073RFRE,GSL074DHAR,GSL074RKEP,GSL075RBER,GSL076DHAW,GSL076RNOR,GSL077DNOV,GSL077RFIT,GSL078RFRA,GSL079DSMI,GSL079RPIG,GSL080RBEA,GSL081DROM,GSL081RSAR,GSL082DCAF,GSL082RTON,GSL083DABB,GSL083RLEA,GSL084DCOH,GSL084RRUC,GSL085DSTO,GSL085RBIN,GSL086DBRO,GSL086RBED,GSL087IMOO,GSL087RMAR,GSL088DMOU,GSL088RRUS,GSL089DSUL,GSL089RWAN,GSL090DMAR,GSL090GMAR,GSL090LHIN,GSL090RAUL,GSL091DKEO,GSL091RWAN,GSL092DSTE,GSL092RHOP,GSL093DAZZ,GSL093LBRA,GSL093RREA,GSL094DFRA,GSL094RMAP,GSL095DHOW,GSL095LWIN,GSL095RSUB,GSL096DKAR,GSL096RWHI,GSL097DJAC,GSL097RHEL,GSL098DCAR,GSL098LARM,GSL098RSHE,GSL099DTHA,GSL099RDAR,GSL100DZEP,GSL100LVAN,GSL100RMCC,GSU01RCUF,GSU04DWAT,GSU04RFUL,GSU05RNOL,GSU08DWEB,GSU08RJEN,GSU09RGIL,GSU11DJAC,GSU11REMR,GSU12DBAC,GSU12RMCK,GSU13DSCH,GSU13RTRE,GSU14DBRE,GSU14RTEM,GSU15DPAL,GSU15RBAR,GSU19RBOG,GSU20RUSH,GSU22DDEN,GSU22RZOL,GSU24DKEL,GSU24RTID,GSU27RLEN,GSU29RMAN,GSU30DISA,GSU30RESP,GSU32DFLO,GSU32RCHA,GSU33DHAY,GSU33RDEW,GSU34DLYN,GSU34RVAN,GSU39DMUL,GSU39RVER,GSU41DELL,GSU41RGAL,GSU42DDUN,GSU42ROLS,GSU43DSCH,GSU43RELL,GSU48DMOR,GSU48RHOR,GSU49DCUR,GSU49RTSC,GSU50DOLS,GSU50RKNO,county,COUNTYNM
0,1-:- 1,1,3,18,99,0,0,0,56,66,17,85,63,58,0,0,0,29,22,2,76,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,43,84,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30003,Big Horn County
1,1-:- 10,10,3,8,45,0,0,0,28,24,12,36,37,15,0,0,0,5,10,1,37,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,44,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30003,Big Horn County


In [27]:
#Column dictionary for readme
dfcol#["new_col"]
readme_column_dict = pd.Series(dfcol["og_column"].str.replace("_", " ").values, index = dfcol["new_col"]).to_dict()

In [28]:
#Readme content

def create_fields_table(race_field_header_0, fields_dict_0, 
                        add_race_field_header_1 = '', fields_dict_1 = {}, 
                        add_race_field_header_2 = '', fields_dict_2 = {}, 
                        add_race_field_header_3 = '', fields_dict_3 = {}):
    '''Purpose: Create fields table used in readme based on field dictionary created separately
    Arguments:
        race_field_header_0: include asterisks "***text***" and label first set of fields
        fields_dict_0: the default dictionary for the primary file (statewide)
        add_race_field_header_1: include asterisks to draw attention to section - ex: "***additional_race_file_name_fields***"
        fields_dict_1: additional fields to go under add_race_field_header_1 header
        add_race_field_header_2 and _3: same use as add_race_field_header_1 - include as needed
        fields_dict_2 and _3: same use as fields_dict_1 - include as needed associated with corresponding add_race_field_header section
    '''
    fields_table_data = {'Field Name': ['',race_field_header_0]  + list(fields_dict_0.keys()) +
                         ['',add_race_field_header_1] + list(fields_dict_1.keys()) +
                         ['',add_race_field_header_2] + list(fields_dict_2.keys()) +
                         ['',add_race_field_header_3] + list(fields_dict_3.keys()),
                         'Description': ['',''] + list(fields_dict_0.values()) + 
                         ['',''] + list(fields_dict_1.values()) + 
                         ['',''] + list(fields_dict_2.values()) +
                         ['',''] + list(fields_dict_3.values())}
    fields_table = pd.DataFrame(fields_table_data)
    return fields_table


#create_fields_table('Field Name: ', readme_column_dict)

fields_table = create_fields_table('', readme_column_dict).sort_values(["Field Name"])

readme_fields = fields_table.to_string(formatters={'Description':'{{:<{}s}}'.format(fields_table['Description'].str.len().max()).format, 'Field Name':'{{:<{}s}}'.format(fields_table['Field Name'].str.len().max()).format}, index=False)

print(readme_fields)

  Field Name                                                              Description
                                                                                     
                                                                                     
                                                                                     
                                                                                     
                                                                                     
                                                                                     
                                                                                     
                                                                                     
G22A48NO     NO   CONSTITUTIONAL AMENDMENT NO. 48                                    
G22A48YES    YES   CONSTITUTIONAL AMENDMENT NO. 48                                   
G22PSC1RPIN  RANDY PINOCCI  REPUBLICAN   PUBLIC SERVIC

In [157]:
final.to_csv("./mt_draft.csv", index=False)